# Explore Data In Redshift

In [ ]:
!pip install -q SQLAlchemy==1.3.13

In [ ]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

redshift = boto3.client('redshift')
secretsmanager = boto3.client('secretsmanager')

# Set Redshift Connection Parameters

In [ ]:
redshift_schema = 'redshift'
redshift_cluster_identifier = 'dsoaws'
redshift_host = 'dsoaws'
redshift_database = 'dsoaws'
redshift_port = '5439'
redshift_table_2015 = 'amazon_reviews_tsv_2015'
redshift_table_2014 = 'amazon_reviews_tsv_2014'

# Load the Redshift Secrets from Secrets Manager

In [ ]:
import json

secret = secretsmanager.get_secret_value(SecretId='dsoaws_redshift_login')
cred = json.loads(secret['SecretString'])

redshift_username = cred[0]['username']
redshift_pw = cred[1]['password']

# Please Wait for Cluster Status  `Available`

In [ ]:
import time
redshift = boto3.client('redshift')

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

# Get Redshift Endpoint Address

In [ ]:
redshift_endpoint_address = response['Clusters'][0]['Endpoint']['Address']

print(redshift_endpoint_address)

# Create the Redshift Query Engine 

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(redshift_username, redshift_pw, redshift_endpoint_address, redshift_port, redshift_database))

# Prepare For Visualizations

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'


# Set Seaborn parameters in advance

In [ ]:
sns.set_style = 'seaborn-whitegrid'

sns.set(rc={"font.style":"normal",
            "axes.facecolor":"white",
            'grid.color': '.8',
            'grid.linestyle': '-',
            "figure.facecolor":"white",
            "figure.titlesize":20,
            "text.color":"black",
            "xtick.color":"black",
            "ytick.color":"black",
            "axes.labelcolor":"black",
            "axes.grid":True,
            'axes.labelsize':10,
            'figure.figsize':(10.0, 10.0),
            'xtick.labelsize':10,
            'font.size':10,
            'ytick.labelsize':10})

# Custom code to display values on bars

In [ ]:
def show_values_barplot(axs, space):
    def _show_on_plot(ax):
        for p in ax.patches:
            _x = p.get_x() + p.get_width() + float(space)
            _y = p.get_y() + p.get_height()
            value = round(float(p.get_width()),2)
            ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_plot(ax)
    else:
        _show_on_plot(axs)

# 1. Which product categories had the most reviews in 2015?

In [ ]:
statement = """
SELECT
    year,
    product_category,
    COUNT(star_rating) AS count_star_rating   
FROM
    {}.{}  
GROUP BY
    product_category,
    year  
ORDER BY
    count_star_rating DESC,
    year DESC
""".format(redshift_schema, redshift_table_2015)

print(statement)

In [ ]:
df = pd.read_sql_query(statement, engine)

In [ ]:
df

In [ ]:
# Store counts
count_ratings = df['count_star_rating']

# Store number of categories
num_categories = df['product_category'].count()
print(num_categories)

# Store max ratings
max_ratings = df['count_star_rating'].max()
print(max_ratings)

# Store min ratings
min_ratings = df['count_star_rating'].min()
print(min_ratings)

# Visualization

In [ ]:
# Create Seaborn barplot
barplot = sns.barplot(y='product_category', x='count_star_rating', data = df, saturation=1)

# Scale if needed
if num_categories < 10:
        sns.set(rc={'figure.figsize':(10.0, 5.0)})
    
# Set title
barplot.set_title("Number of Ratings Per Product Category (2015)", fontsize="20")

# Set x-axis ticks to match scale 
if max_ratings <= 200000:
    plt.xticks([50000, 100000, 150000, 200000], ['50K', '100K', '1500K', '200K'])
    plt.xlim(0, 200000)
elif max_ratings <= 5000000:
    plt.xticks([1000000, 2000000, 3000000, 4000000, 5000000], ['1m', '2m', '3m', '4m', '5m'])
    plt.xlim(0, 5000000)
else:
    plt.xticks([100000, 1000000, 5000000, 10000000, 15000000, 20000000], ['100K', '1m', '5m', '10m','15m','20m'])
    plt.xlim(0, 20000000)

plt.xlabel("Number of Ratings", fontsize="15")
plt.ylabel("Product Category", fontsize="15")

plt.tight_layout()

# Export plot if needed
# plt.savefig('redshift_2015_ratings_per_category.png', dpi=300)

# Show the barplot
plt.show(barplot)

# Visualization for All Product Categories (150+ Million Reviews)
If you ran this same query across all product categories (150+ million reviews), you would see the following visualization:

<img src="img/c4-09.png"  width="80%" align="left">

# 2. Which products have the most helpful reviews in 2015?
How long are those reviews?

In [ ]:
statement = """
SELECT
    product_title,
    helpful_votes,
    LENGTH(review_body) AS review_body_length,
    SUBSTRING(review_body, 1, 100) AS review_body_substring 
FROM
    {}.{} 
ORDER BY
    helpful_votes DESC LIMIT 10 
""".format(redshift_schema, redshift_table_2015)

print(statement)

In [ ]:
df = pd.read_sql_query(statement, engine)

In [ ]:
df

# Results for All Product Categories (150+ Million Reviews)
If you ran this same query across all product categories (150+ million reviews), you would see the following result:

<img src="img/most_helpful_2015.png"  width="90%" align="left">

# 3. What is the breakdown of star ratings (1-5) per product category in 2015?

In [ ]:
statement = """
SELECT
    product_category,
    star_rating,
    COUNT(DISTINCT review_id) AS count_reviews 
FROM
    {}.{} 
GROUP BY
    product_category,
    star_rating 
ORDER BY
    product_category ASC,
    star_rating DESC,
    count_reviews
""".format(redshift_schema, redshift_table_2015)

print(statement)

In [ ]:
df = pd.read_sql_query(statement, engine)

In [ ]:
df

#### Which product categories are the highest rated by average rating?

In [ ]:
# SQL statement
statement = """
SELECT
    product_category,
    COUNT(star_rating) AS count_stars,
    AVG(star_rating::FLOAT) AS avg_star_rating
FROM
    {}.{}  
GROUP BY
    product_category  
ORDER BY
    avg_star_rating DESC
""".format(redshift_schema, redshift_table_2015)

print(statement)

In [ ]:
df_average_rating = pd.read_sql_query(statement, engine)

In [ ]:
df_average_rating

# Stacked percentage horizontal bar plot showing proportion of star ratings per product category

In [ ]:
# Create grouped DataFrames by category and by star rating
grouped_category = df.groupby('product_category')
grouped_star = df.groupby('star_rating')

# Create sum of ratings per star rating
df_sum = df.groupby(['star_rating']).sum()

# Calculate total number of star ratings
total = df_sum['count_reviews'].sum()
print(total)

In [ ]:
# Create dictionary of product categories and array of star rating distribution per category
distribution = {}
count_reviews_per_star = []
i=0
    
for category, ratings in grouped_category:
    count_reviews_per_star = []
    for star in ratings['star_rating']:
        count_reviews_per_star.append(ratings.at[i, 'count_reviews'])
        i=i+1;
    distribution[category] = count_reviews_per_star

# Check if distribution has been created succesfully
print(distribution)

In [ ]:
# Check if distribution keys are set correctly to product categories
print(distribution.keys())

In [ ]:
# Check if star rating distributions are set correctly
print(distribution.items())

# Build array per star across all categories

In [ ]:
# Sort distribution by highest average rating per category
sorted_distribution = {}

df_average_rating.iloc[:,0]
for index, value in df_average_rating.iloc[:,0].items():
    sorted_distribution[value] = distribution[value]

sorted_distribution

In [ ]:
# Build array per star across all categories
star1 = []
star2 = []
star3 = []
star4 = []
star5 = []

for k in sorted_distribution.keys():
    stars = sorted_distribution.get(k)
    star5.append(stars[0])
    star4.append(stars[1])
    star3.append(stars[2])
    star2.append(stars[3])
    star1.append(stars[4])

# Visualization

In [ ]:
categories = sorted_distribution.keys()

total = np.array(star1) + np.array(star2) + np.array(star3) + np.array(star4) + np.array(star5)

proportion_star1 = np.true_divide(star1, total) * 100
proportion_star2 = np.true_divide(star2, total) * 100
proportion_star3 = np.true_divide(star3, total) * 100
proportion_star4 = np.true_divide(star4, total) * 100
proportion_star5 = np.true_divide(star5, total) * 100

# Add colors
colors = ['red', 'purple','blue','orange','green']

# The position of the bars on the x-axis
r = range(len(categories))
barHeight = 1

# Plot bars
if num_categories > 10:
    plt.figure(figsize=(10,10))
else: 
    plt.figure(figsize=(10,5))

ax5 = plt.barh(r, proportion_star5, color=colors[4], edgecolor='white', height=barHeight, label='5-Star Ratings')
ax4 = plt.barh(r, proportion_star4, left=proportion_star5, color=colors[3], edgecolor='white', height=barHeight, label='4-Star Ratings')
ax3 = plt.barh(r, proportion_star3, left=proportion_star5+proportion_star4, color=colors[2], edgecolor='white', height=barHeight, label='3-Star Ratings')
ax2 = plt.barh(r, proportion_star2, left=proportion_star5+proportion_star4+proportion_star3, color=colors[1], edgecolor='white', height=barHeight, label='2-Star Ratings')
ax1 = plt.barh(r, proportion_star1, left=proportion_star5+proportion_star4+proportion_star3+proportion_star2, color=colors[0], edgecolor='white', height=barHeight, label="1-Star Ratings")

plt.title("Distribution of Reviews Per Rating Per Category (2015)",fontsize='16')
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.yticks(r, categories, fontweight='regular')

plt.xlabel("% Breakdown of Star Ratings", fontsize='14')
plt.gca().invert_yaxis()
plt.tight_layout()

# plt.savefig('redshift_2015_proportion_star_per_category.png', dpi=300)
plt.show()

# Visualization for All Product Categories (150+ Million Reviews)
If you ran this same query across all product categories (150+ million reviews), you would see the following visualization:

<img src="img/c4-10.png"  width="70%" align="left">

# 4. How did the star ratings change during 2015?
Is there a drop-off point for certain product categories throughout the year?

In [ ]:
statement = """
SELECT
    CAST(DATE_PART('month', TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS month,
    AVG(star_rating::FLOAT) AS avg_rating  
FROM
    {}.{} 
GROUP BY
    month
ORDER BY
    month
""".format(redshift_schema, redshift_table_2015)

print(statement)

In [ ]:
df = pd.read_sql_query(statement, engine)
df

# Visualization

In [ ]:
fig = plt.gcf()
fig.set_size_inches(12,5)

fig.suptitle('Average Star Rating Throughout 2015 (Across Subset Of Product Categories)')

ax = plt.gca()

ax.locator_params(integer=True)
ax.set_xticks(df['month'].unique())

df.plot(kind='line',x='month',y='avg_rating', color='red', ax=ax)

plt.xlabel('Months')
plt.ylabel('Average Star Rating')

# fig.savefig('redshift_2015_average_rating.png', dpi=300)
plt.show()

# Visualization for All Product Categories (150+ Million Reviews)
If you ran this same query across all product categories (150+ million reviews), you would see the following visualization:

<img src="img/c4-11.png"  width="80%" align="left">

## Is there a drop-off point for certain product categories throughout the year?

In [ ]:
statement = """
SELECT
    product_category,
    CAST(DATE_PART('month', TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS month,
    AVG(star_rating::FLOAT) AS avg_rating  
FROM
    {}.{} 
GROUP BY
    product_category, month
ORDER BY
    product_category, month
""".format(redshift_schema, redshift_table_2015)

print(statement)

In [ ]:
df = pd.read_sql_query(statement, engine)
df

In [ ]:
def plot_categories(df):
    df_categories = df['product_category'].unique()
    for category in df_categories:
        # print(category)
        df_plot = df.loc[df['product_category'] == category]
        df_plot.plot(kind='line',x='month',y='avg_rating', c=np.random.rand(3,), ax=ax, label=category)
        

# Visualization

In [ ]:
fig = plt.gcf()
fig.set_size_inches(12,5)

fig.suptitle('Average Star Rating Over Time Per Category In 2015')
           
ax = plt.gca()

ax.locator_params(integer=True)
ax.set_xticks(df['month'].unique())

plot_categories(df)

plt.xlabel('Month')
plt.ylabel('Average Star Rating')
plt.legend(bbox_to_anchor=(0, -0.15, 1, 0), loc=2, ncol=2, mode="expand", borderaxespad=0)

# fig.savefig('redshift_2015_average_rating_category.png', dpi=300)
plt.show()

# Visualization for All Product Categories (150+ Million Reviews)
If you ran this same query across all product categories (150+ million reviews), you would see the following visualization:

![](img/redshift_2015_average_rating_category.png)

# 5. Which customers are abusing the review system in 2015 by repeatedly reviewing the same product more than once?  
What was their average star rating for each product?


In [ ]:
# SQL statement 
statement = """
SELECT
    customer_id,
    product_category,
    product_title,
    ROUND(AVG(star_rating::FLOAT), 4) AS avg_star_rating,
    COUNT(*) AS review_count  
FROM
    redshift.amazon_reviews_tsv_2015 
GROUP BY
    customer_id,
    product_category,
    product_title  
HAVING
    COUNT(*) > 1  
ORDER BY
    review_count DESC LIMIT 5 
""".format(redshift_schema, redshift_table_2015)

print(statement)

In [ ]:
df = pd.read_sql_query(statement, engine)
df

# Visualization for All Product Categories (150+ Million Reviews)
If you ran this same query across all product categories (150+ million reviews), you would see the following visualization:

<img src="img/redshift-abuse-all.png"  width="80%" align="left">

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();